In [1]:
import numpy as np
import pandas as pd

In [33]:
d = {'Clubes': [], 'PG': [], 'j': [], 'V': [], 'E': [], 'D': [], 'GP': [], 'GC': [], 'SG': []}
dfClassChampionship = pd.DataFrame(data=d);
dfDataChampionship = pd.read_excel("b2005.xlsx", encoding='utf8')

In [34]:
dfDataChampionship.head()

,Horario,Dia,Data,Clube 1,Clube 2,Placar,Vencedor,Rodada,Arena
0,16h00,Sábado,23/04/2005,Paraná,Goiás,0x2,Goiás,1ª Rodada,Pinheirão
1,16h00,Sábado,23/04/2005,Atlético-MG,Figueirense,4x1,Atlético-MG,1ª Rodada,Mineirão
2,18h10,Sábado,23/04/2005,São Caetano,Palmeiras,2x2,-,1ª Rodada,Bruno J Daniel
3,18h10,Sábado,23/04/2005,Flamengo,Cruzeiro,1x1,-,1ª Rodada,Maracanã
4,16h00,Domingo,24/04/2005,Internacional,Botafogo,0x2,Botafogo,1ª Rodada,Beira Rio


** Get all clubes for championship **



In [35]:
#Cancatenando as colunas Clube 1 e Clube 2
clubs  =  pd.concat([dfDataChampionship['Clube 1'].str.lower(), dfDataChampionship['Clube 2'].str.lower()], axis=1, keys=['Clubes'])

In [36]:
#Obtem todos os clubes 
cb = pd.DataFrame({'Clubes': clubs['Clubes'].unique()})

In [37]:
#Todos os clubes que participaram dp campeonato
cb

,Clubes
0,paraná
1,atlético-mg
2,são caetano
3,flamengo
4,internacional
5,fortaleza
6,fluminense
7,corinthians
8,atlético-pr
9,brasiliense


In [38]:
def getNumberWinner(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter = df["Vencedor"].str.lower() == clube
    return (data[filter]['Vencedor'].count()).astype(np.int64)

def getNumberDepartures(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    return (data[filter1]['Clube 1'].count() + data[filter2]['Clube 2'].count()).astype(np.int64)

def getPoints(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = df["Clube 1"].str.lower() == clube
    filter2 = df["Clube 2"].str.lower() == clube
    filter3 = df["Vencedor"].str.lower() == clube
    filter4 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter5 = df["Vencedor"].str.lower() == '-'
    
    v1 = data[(filter1) & (filter3)]
    v1 = v1['Vencedor'].count()
    v2 = data[(filter2) & (filter3)]
    v2 = v2['Vencedor'].count() 
    v3 = data[(filter4) & (filter5)]
    v3 = v3['Vencedor'].count()
    return ((v2*3)+(v1*3)+v3).astype(np.int64)

def getDraw(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = df["Vencedor"].str.lower() == '-'
    df = data[(filter1) & (filter2)]
    empates = df['Vencedor'].count()
    return empates.astype(np.int64)

def getDefeats(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1 = (df["Clube 1"].str.lower() == clube) | (df["Clube 2"].str.lower() == clube)
    filter2 = (df["Vencedor"].str.lower() != clube) & (df["Vencedor"].str.lower() != '-')
    df = data[(filter1) & (filter2)]
    derrotas = df['Vencedor'].count()
    return derrotas.astype(np.int64)

def getGP(data, clube ):
    df = data.apply(lambda x: x.str.strip())
    filter1  =  df["Clube 1"].str.lower() == clube
    filter2  =  df["Clube 2"].str.lower() == clube
    df1      =  data[(filter1)]
    df2      =  data[(filter2)]
    placar1  =  df1['Placar'].str.split('x')
    placar2  =  df2['Placar'].str.split('x')
    gp       =  0
    gc       =  0
    for g1, g2 in placar1:
        gp = (gp + pd.to_numeric( g1 ))
        gc = (gc + pd.to_numeric( g2 ))
            
    for g1, g2 in placar2:
        gp = (gp + pd.to_numeric( g2 )) 
        gc = (gc + pd.to_numeric( g1 ))
    return gp, gc

In [51]:
dfTable = cb[["Clubes"]].copy()
for column in ["PG", "J", "V", "E", "D", "GP", "GC", "SG"]:
    dfTable[column] = pd.np.nan
    
    
for index, row in dfTable.iterrows():
    c1  =  row['Clubes']
    if type(c1) != unicode:
        c1 =  c1.decode('utf-8')
    c1 =  c1.strip()
    
    pg       =  getPoints(dfDataChampionship, c1 )
    j        =  getNumberDepartures(dfDataChampionship, c1 )
    v        =  getNumberWinner(dfDataChampionship, c1 )
    e        =  getDraw(dfDataChampionship, c1 )
    d        =  getDefeats(dfDataChampionship, c1 )
    gp, gc   =  getGP(dfDataChampionship, c1 )
    
    if c1 == u'vasco':
        pg  += 2 #Vasco venceu o processo na justica e ganhou os pontos da partida contra o brasiliense
        v   += 1
        e   -= 1
        gp  -= 1
        gc  -= 2
    elif c1 == u'brasiliense':
        pg  -= 1 #Foi punido pela justiça
        e   -= 1
        d   += 1
        gc  -= 1
        gp  -= 2
        
    dfTable.at[index, 'PG'] = pg
    dfTable.at[index, 'J']  = j
    dfTable.at[index, 'V']  = v
    dfTable.at[index, 'E']  = e
    dfTable.at[index, 'D']  = d
    dfTable.at[index, 'GP'] = gp
    dfTable.at[index, 'GC'] = gc
    dfTable.at[index, 'SG'] = gp - gc
    
    #print c1

dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
0,paraná,61.0,42.0,17.0,10.0,15.0,59.0,51.0,8.0
1,atlético-mg,47.0,42.0,13.0,8.0,21.0,54.0,59.0,-5.0
2,são caetano,52.0,42.0,14.0,10.0,18.0,54.0,60.0,-6.0
3,flamengo,55.0,42.0,14.0,13.0,15.0,56.0,60.0,-4.0
4,internacional,78.0,42.0,23.0,9.0,10.0,72.0,49.0,23.0
5,fortaleza,55.0,42.0,16.0,7.0,19.0,58.0,64.0,-6.0
6,fluminense,68.0,42.0,19.0,11.0,12.0,79.0,70.0,9.0
7,corinthians,81.0,42.0,24.0,9.0,9.0,87.0,59.0,28.0
8,atlético-pr,61.0,42.0,18.0,7.0,17.0,76.0,67.0,9.0
9,brasiliense,41.0,42.0,10.0,11.0,21.0,47.0,67.0,-20.0


In [52]:
reorderlist = [ 'Corinthians', 'Internacional', 'Goiás', 'Palmeiras', 'Fluminense', 'Atlético-pr', 
                'Paraná', 'Cruzeiro', 'Botafogo', 'Santos', 'São Paulo', 'Vasco', 'Fortaleza', 
                'Juventude', 'Flamengo', 'Figueirense', 'São Caetano', 'Ponte Preta', 'Coritiba', 
                'Atlético-mg', 'Paysandu', 'Brasiliense']
listIndex   = []

#Processa a lista para gerar a lista de ordenação
for clube in reorderlist:
    for index, row in dfTable.iterrows():
        #Verifica se o valor de clube é um unicode, se não, convete isso.
        if type(clube) != unicode:
            c =  clube.decode('utf-8')
        else:
            c =  clube
        
        c1  =  row['Clubes']
        #print c1
        if type(c1) != unicode:
            c1 =  c1.decode('utf-8')
        
        c1 =  c1.strip()
        c  =  c.lower()
        
        #Verifica se o clube iterado é igual ao clube da lista, se sim
        #retorna seu indice
        if c1 == c:
            listIndex.append( index )
            break
            
#Reordena a lista
dfTable  =  dfTable.reindex(index=listIndex, columns=["Clubes", "PG", "J", "V", "E", "D", "GP", "GC", "SG"])
#Com a lista ordenada, corrige os indices.
dfTable.index =  np.arange(1, len(dfTable)+1)
dfTable

,Clubes,PG,J,V,E,D,GP,GC,SG
1,corinthians,81.0,42.0,24.0,9.0,9.0,87.0,59.0,28.0
2,internacional,78.0,42.0,23.0,9.0,10.0,72.0,49.0,23.0
3,goiás,74.0,42.0,22.0,8.0,12.0,68.0,51.0,17.0
4,palmeiras,70.0,42.0,20.0,10.0,12.0,81.0,65.0,16.0
5,fluminense,68.0,42.0,19.0,11.0,12.0,79.0,70.0,9.0
6,atlético-pr,61.0,42.0,18.0,7.0,17.0,76.0,67.0,9.0
7,paraná,61.0,42.0,17.0,10.0,15.0,59.0,51.0,8.0
8,cruzeiro,60.0,42.0,17.0,9.0,16.0,73.0,72.0,1.0
9,botafogo,59.0,42.0,17.0,8.0,17.0,57.0,56.0,1.0
10,santos,59.0,42.0,16.0,11.0,15.0,68.0,71.0,-3.0
